In [1]:
import pandas as pd
import tqdm
import folium
from folium import plugins

In [2]:
TRAM_ICON = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTo8ghk-iBncj_HnWVMjR623u97xyVwnGLmPg&usqp=CAU"

In [3]:
# disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [4]:
VEHICLE_STOPS_FILE_NAME = 'przystanki_2022-10-22.pkl'
TIMETABLES_FILE_NAME = 'rozklady_2022-10-22.pkl'

In [5]:
df = pd.read_pickle(VEHICLE_STOPS_FILE_NAME, compression='zip')
df_full = pd.read_pickle(TIMETABLES_FILE_NAME, compression='zip')

In [6]:
def generate_all_vehicle_numbers(df_full: pd.DataFrame, vehicle_type: str = None) -> list:
    '''
    Utwóz listę numerów wszystkich pojazdów

    Arguments:
        df_full: tabela z pełnymi danymi przystankowymi, numerami linii, brygadami i czasami
        vehicle_type: typ pojazdu. Możliwe opcje:
            T - tramwaj
            A - autobus
            M - metro
            S - pociag SKM 
            R - pociąg KM 
            WKD - pociąg WKD 
            None - wszystkie typy pojazdów
        
    Returns:
        Lista unikalnyh numerów pojazdów
    '''
    if vehicle_type == None:
        return df_full[df_full['brygada'].apply(lambda x: len(x)>0)]['linie'].unique().tolist()
    else:
       return  df_full[(df_full['typ'] == vehicle_type) & (df_full['brygada'].apply(lambda x: len(x)>0))]['linie'].unique().tolist()

In [7]:
lines_list = generate_all_vehicle_numbers(df_full, None)

In [8]:
lines_list

['102',
 '123',
 '125',
 '138',
 '146',
 '147',
 '166',
 '202',
 '509',
 'N02',
 'N03',
 'N21',
 'N71',
 '3',
 '6',
 '7',
 '9',
 '22',
 '24',
 '26',
 '13',
 '28',
 '78',
 '120',
 '169',
 '512',
 '135',
 '162',
 '170',
 'N14',
 'N16',
 'N64',
 '4',
 '20',
 '23',
 '160',
 '190',
 'N11',
 'N61',
 '212',
 '409',
 '500',
 'N12',
 'N62',
 '1',
 '211',
 '516',
 '705',
 '723',
 '731',
 '736',
 '112',
 '114',
 '132',
 '134',
 '145',
 '156',
 '186',
 '735',
 'N13',
 'N63',
 '518',
 '126',
 '511',
 '152',
 '133',
 '900',
 'L49',
 '233',
 '140',
 '199',
 '245',
 '738',
 '104',
 '204',
 '226',
 '269',
 '176',
 '234',
 'N01',
 '2',
 '17',
 '240',
 '256',
 '527',
 'L10',
 'L34',
 'L41',
 'L-9',
 'L11',
 'L45',
 'L40',
 'L43',
 'L46',
 'L36',
 'L35',
 'L37',
 'L38',
 'R9',
 'S3',
 'RL',
 'R6',
 '521',
 'N24',
 '173',
 '188',
 '523',
 '141',
 '143',
 '142',
 '702',
 '704',
 '720',
 '722',
 '730',
 'Z42',
 '183',
 '148',
 'N22',
 '502',
 '514',
 '520',
 '525',
 'N25',
 '115',
 '119',
 '213',
 '161',
 'N

In [9]:
def make_timetables_dict(df_full: pd.DataFrame, lines_list: list) -> dict:
    '''
    Generuje trasy na podstawie pobranych informacji.

    Arguments:
        lines_list: lista unikalnych numerów linii
        df_full: tabela z pełnymi danymi przystankowymi, numerami linii, brygadami i czasami
    Returns:
        Słownik w następującej postaci:
        {'102': {'57_TP-OLS': 
                    [('05:02', '1231_07'),
                    ('05:03', '1232_04'),
                    ('05:04', '1231_02'),
                    ('05:06', '1001_01'),
                    ('05:07', '2001_04'),
                    ...
    '''
    lines_dict = {}
    for line in tqdm.tqdm(lines_list):
        df_test = df_full[df_full['linie'] == line]
        df_test['concat'] = None
        d = {}
        for index, row in df_test.iterrows():
            df_test.at[index, 'concat'] = {a:x +'_'+ y for (a, x, y) in zip(df_test['czas'][index], df_test['brygada'][index], df_test['trasa'][index])}
            for k,v in df_test.at[index, 'concat'].items():
                d.setdefault(v, []).append((k, df_test.at[index, 'zespol']+'_'+df_test.at[index, 'slupek'])) 

        for bt in d:
            d[bt] = sorted(list(d.values())[0], key = lambda x: x[0])
        lines_dict[line] = d
    return lines_dict

In [10]:
lines_dict = make_timetables_dict(df_full, lines_list)

100%|██████████| 295/295 [00:11<00:00, 25.54it/s]


In [ ]:
lines_dict

In [12]:
def get_routes(line_number, lines_dict):
    result = []
    for elem in lines_dict[line_number]:
        result.append(elem)
    return dict(zip(range(len(result)), result))

In [13]:
get_routes('709', lines_dict)

{0: '3_TP-PIP',
 1: '5_TP-PIP',
 2: '1_TP-PIP',
 3: '2_TP-PIP',
 4: '4_TP-PIP',
 5: '6_TP-PIP',
 6: '64_TX-PIS',
 7: '62_TX-PIS',
 8: '66_TX-PIS',
 9: '1_TP-WIL-P',
 10: '3_TP-WIL-P',
 11: '5_TP-WIL-P',
 12: '2_TP-WIL-P',
 13: '4_TP-WIL-P',
 14: '6_TP-WIL-P',
 15: '64_TX-WIL17',
 16: '62_TX-WIL17',
 17: '66_TX-WIL17',
 18: '3_TX-WIL02',
 19: '5_TX-WIL02',
 20: '6_TX-WIL02'}

In [14]:
def get_schedule(line_number, route_order, lines_dict):
    routes = get_routes(line_number, lines_dict)
    return routes[route_order], lines_dict[line_number][routes[route_order]]

In [ ]:
get_schedule('709', 9, lines_dict)

In [16]:
def get_full_schedule(line_number, route_order, df, lines_dict):
    df_times = pd.DataFrame(get_schedule(line_number, route_order, lines_dict)[1])[0]
    df_places = pd.DataFrame(get_schedule(line_number, route_order, lines_dict)[1])[1].str.split('_', expand=True)
    d = pd.concat([df_times, df_places], axis=1)
    d.columns = ['czas', 'zespol', 'slupek']
    return pd.merge(d, df, how='left', right_on=['zespol', 'slupek'], left_on=['zespol', 'slupek'])

---

## Podsumowując ##

Ładujemy tylko raz

In [17]:
#Ładujemy tylko raz:
VEHICLE_STOPS_FILE_NAME = 'przystanki_2022-10-22.pkl'
TIMETABLES_FILE_NAME = 'rozklady_2022-10-22.pkl'

df = pd.read_pickle(VEHICLE_STOPS_FILE_NAME, compression='zip')
df_full = pd.read_pickle(TIMETABLES_FILE_NAME, compression='zip')

lines_list = generate_all_vehicle_numbers(df_full, None)
lines_dict = make_timetables_dict(df_full, lines_list)

100%|██████████| 295/295 [00:11<00:00, 26.52it/s]


Ładujemy za każdym razem dla linii, którą chcemy sprawdzić

In [18]:
LINE_NUMBER = '709'
ROUTE_ORDER = 0
df_routes = get_full_schedule(LINE_NUMBER, ROUTE_ORDER, df, lines_dict)

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_routes)

,czas,zespol,slupek,nazwa_zespolu,id_ulicy,szer_geo,dlug_geo,kierunek,obowiazuje_od,linie
0,04:25,3009,18,Metro Wilanowska,0125,52.179943,21.024275,Wałbrzyska,2022-06-10 00:00:00.0,"[709, 727, 739]"
1,04:29,3011,01,al.Lotników,1801,52.172310,21.019512,Wyścigi,2022-06-10 00:00:00.0,"[192, 331, 709, 727, 739, N83]"
2,04:30,3012,01,Wyścigi,1801,52.164329,21.017053,Poleczki,2022-06-10 00:00:00.0,"[192, 331, 709, 727, 739, N83]"
3,04:32,3013,01,Poleczki,1801,52.151950,21.017323,Grabów,2022-06-10 00:00:00.0,"[709, 727, 739, N83, N86]"
4,04:33,3014,01,Grabów,1801,52.146720,21.017450,Sójki,2022-06-10 00:00:00.0,"[709, 727, 739, N83, N86]"
5,04:34,3015,01,Sójki,1801,52.143050,21.017560,Żołny,2022-06-10 00:00:00.0,"[209, 709, 715, 727, 737, 739, 809, 815, N83]"
6,04:35,3016,01,Żołny,1801,52.138447,21.017487,Bogatki,2022-06-10 00:00:00.0,"[209, 709, 715, 727, 737, 739, 809, 815, N83]"
7,04:37,3017,01,Bogatki,1801,52.131876,21.017773,Łagiewnicka,2022-06-10 00:00:00.0,"[209, 709, 715, 727, 737, 739, 809, 815, N83]"
8,04:38,3018,01,Łagiewnicka,1801,52.126961,21.017897,Pelikanów,2022-06-10 00:00:00.0,"[209, 709, 715, 727, 737, 739, 809, 815, N83]"
9,04:39,3019,01,Pelikanów,1801,52.122514,21.018020,Kapeli,2022-06-10 00:00:00.0,"[209, 709, 715, 727, 737, 739, 809, 815, N83]"


Zwizualizuj trasę

In [20]:
# create folium map and set starting point as first point in dataframe
date = VEHICLE_STOPS_FILE_NAME.split('_')[1].split('.')[0]
converted_dates = df_routes['czas'].apply(lambda x: f'{date}T{x}').tolist()

m = folium.Map(location=df_routes[['szer_geo', 'dlug_geo']].iloc[0].tolist(), zoom_start=16)

# add geocoder (textbox to input geolocation names)
plugins.Geocoder().add_to(m)

# draw lines based on coordinates in dataframe
# 'lon' is first, 'lat' is second
lines = [
    {
        "coordinates": [[x,y] for x,y in zip(df_routes['dlug_geo'], df_routes['szer_geo'])],
        "dates": converted_dates,
        "color": "red"}]

# make 'features' to output them into TimestampedGeoJson
features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"],
                },
            "icon": "marker",

            # comment whole 'iconstyle' for a default marker
            "iconstyle": { 
                "iconUrl": TRAM_ICON,
                "iconSize": [40, 40],
            },
        },
    }
    for line in lines
]

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1M",
    add_last_point=True,
).add_to(m)

In [21]:
m.save(f'{LINE_NUMBER}_{get_routes(LINE_NUMBER, lines_dict)[ROUTE_ORDER]}.html')